# Modelo VRP con Open Route Service y Google OR-Tools
#### OR-Tools + ORS
* https://developers.google.com/optimization/routing/vrp
* https://github.com/googlemaps/google-maps-services-python
* https://jupyter-gmaps.readthedocs.io/en/latest/index.html
* https://github.com/GIScience/openrouteservice-py
* https://openrouteservice.org/
* https://openrouteservice-py.readthedocs.io/en/latest/

## SMART WCVRP (Waste Collection Vehicle Routing Problem)

En el presente notebook se presenta la implementación de un modelo de optimización de recolección de residuos sólidos (WCVRP) utilizando las librerías Open Route Service y VROOM, teniendo presentes los datos obtenidos en tiempo real de contenedores con dispositivos IoT.

El objetivo principal del modelo es realizar la optimización de la logística de las posibles rutas de los vehículos dispuestos para la recolectar los residuos de los contenedores inteligentes. El modelo tiene en cuenta dos fuentes principales de información:

* **Datos de contenedores**: se obtienen de los dispositivos IoT, que se encuentran en el sistema de gestión de contenedores dentro de la plataforma FIWARE; datos como el nombre de la ubicación, la posición geográfica del contenedor y el nivel actual de llenado del contenedor.
* **Datos de usuario**: información de configuración del problema específico a resolver, como el número de vehículos, capacidad de los vehículos, horarios de los vehículos, nivel o threshold para filtrar los contenedores que entran en el modelo.

### Instalación de librerías
Primero se instalan las librerías necesarias para poder hacer uso del notebook, en este caso pandas, datetime, openrouteservice, ortools y folium.

In [ ]:
!pip install pandas
!pip install datetime
!pip install openrouteservice
!pip install folium
!pip install ortools

## Configuración de Usuario
En esta etapa se proceden a suministrar los datos de configuración del problema específico a resolver, y se ejecutan los siguientes pasos:

* Configurar llave API de ORS.
* Crear el depósito.
* Configurar el número de vehículos.
* Configurar los datos de los vehículos disponibles.
* Determinar el nivel de filtrado de los contenedores.

### Configurar la llave API de ORS

Para correr el notebook, es necesario usar una llave API de la plataforma de Open Route Service.

In [21]:
import openrouteservice as ors

# Suministrar la API Key de OpenRouteService del usuario
API_KEY = '5b3ce3597851110001cf62482d6296cdeb7c4441a0b91f42ea31b15a'
client = ors.Client(key=API_KEY)

### Crear el Depósito
El primer paso es crear un depósito y las ubicaciones de recogida. El depósito es la casa o base de los vehículos de recogida y las ubicaciones son los puntos de recogida. Estos lugares son determinados con los puntos de contenedores de basura que se tendrían en Fiware.

In [22]:
# Configurar el relleno sanitario
depot = {   'name': 'Relleno Sanitario La Pradera',
            'location': (6.5171946910919, -75.25502818729257)}

### Configurar el Número de Vehículos
El número de vehículos se configura como 3. Después de correr todo el notebook se puede cambiar esta cantidad para observar su impacto en las rutas generadas.

In [23]:
# Configurar el número de vehículos disponibles
num_vehicles = 3

### Configurar los datos de los Vehículos Disponibles
Se crean los datos correspondientes a las restricciones de capacidad de los vehículos recolectores de basura. Se debe tener presente que las unidades de capacidad deben coincidir (capacidad de los vehículos y contenedores, ya sea en kg o en L).

In [ ]:
# configurar capacidad de los vehículos
vehicle_capacities = [100, 100, 100]

### Determinar el nivel de filtrado de los contenedores
Se determina el nivel de filtrado de los contenedores, es decir, el nivel mínimo de llenado que debe tener un contenedor para ser considerado en el modelo.

In [ ]:
# Configurar el nivel de filtrado
threshold_level = 75

### Crear los contenedores
Se crea un diccionario con los datos de cada uno de los contenedores a recoger. Cuando se tengan los dispositivos instalados para los contenedores se buscaría tomar los datos directamente desde Orion con este formato.

In [24]:
contenedores = [
    { 
        'name': 'Plaza Botero',
        'location': (6.2524857541249315, -75.5686261812429),
        'level': 100,
        'weight': 10,
    },
    {
        'name': 'Universidad de Medellín',
        'location': (6.231704691813073, -75.61160925921747),
        'level': 80,
        'weight': 8,
    },
    {
        'name': 'Parque Juanes de la Paz',
        'location': (6.290384323934336, -75.56945687270888),
        'level': 90,
        'weight': 9,
    },
    {
        'name': 'Parque del Poblado',
        'location': (6.210362095508166, -75.57088108434691),
        'level': 75,
        'weight': 7,
    },
    {
        'name': 'Alcaldía de Medellín',
        'location': (6.2451496127526225, -75.57371337731854),
        'level': 75,
        'weight': 7,
    },
    {
        'name': 'Parque Explora',
        'location': (6.271208962002754, -75.56556245591827),
        'level': 90,
        'weight': 9,
    },
    {
        'name': 'Museo de Arte Moderno de Medellín',
        'location': (6.2240110064206, -75.57412303992716),
        'level': 50,
        'weight': 5,
    },
    {
        'name': 'Parque de Belén',
        'location': (6.232689716085326, -75.59665353624818),
        'level': 60,
        'weight': 6,
    }
]


### Graficar mapa con todas las ubicaciones y el depósito
Se crea una gráfica del mapa con open street maps para visualizar todas las ubicaciones de recogida y el depósito.

In [25]:
import folium as fm
from folium import Marker as mk

coordenadas_medellin = (6.25184, -75.56359)
coordenadas_girardota = (6.3792, -75.4453) # Permiten visualizar el mapa con un mejor zoom y en un punto central entre los contenedores de basura y el depósito

m = fm.Map(location=coordenadas_girardota, tiles='openstreetmap', zoom_start=12)

for node in contenedores:
    tooltip = fm.map.Tooltip("<h4><b>{}</b></p><p>Nivel: <b>{}</b></p><p>Peso: <b>{}</b></p>".format(node['name'], node['level'], node['weight']))
    ttdep = fm.map.Tooltip("<h4><b>{}</b></p>".format(depot['name']))
    mk(location=node['location'], tooltip=tooltip, icon=(fm.Icon(color='gray', icon='trash', prefix='fa'))).add_child(fm.Popup('{}'.format(node['name']))).add_to(m)
    

mk(location=depot['location'], tooltip=ttdep, icon= (fm.Icon(color='red', icon='industry', prefix='fa'))).add_child(fm.Popup('{}'.format(depot['name']))).add_to(m)
m

### Crear la función de filtrado
Se crea una función para filtrar los contenedores que van a entrar en el modelo, de acuerdo al nivel captura por cada contenedor. Para esta simulación se define que los contenedores por debajo de un 75% de la capacidad de carga no entran en el modelo.

In [26]:
def filter_containers(contenedores, level):     
    # Se filtran o seleccionan los contenedores que harían parte del modelo de acuerdo al threshold especificado          
    cont_filt = [c for c in contenedores if c['level'] >= level]
    #contenedores_filtrados = list(filter(lambda x:x['level'] >= level, contenedores))
    return cont_filt

def print_containers(contenedores):
    import pandas as pd
    # Se obtiene un dataframe con los contenedores filtrados que harían parte del modelo
    keys = []
    vals = []
    for data in contenedores:
        val = []
        for k,v in data.items():
            keys.append(k)
            val.append(v)
        vals.append(val)
    filtered_df = pd.DataFrame([v for v in vals], columns=list(dict.fromkeys(keys)))
    return filtered_df

contenedores_filtrados = filter_containers(contenedores, threshold_level)
print_containers(contenedores_filtrados)

,name,location,level,weight
0,Plaza Botero,"(6.2524857541249315, -75.5686261812429)",100,10
1,Universidad de Medellín,"(6.231704691813073, -75.61160925921747)",80,8
2,Parque Juanes de la Paz,"(6.290384323934336, -75.56945687270888)",90,9
3,Parque del Poblado,"(6.210362095508166, -75.57088108434691)",75,7
4,Alcaldía de Medellín,"(6.2451496127526225, -75.57371337731854)",75,7
5,Parque Explora,"(6.271208962002754, -75.56556245591827)",90,9


Se crea una variable que contenga las ubicaciones de los contenedores que entran al modelo para obtener la matriz de distancia.

In [27]:
contenedor_locations = [contenedor['location'] for contenedor in contenedores_filtrados]
contenedor_locations

[(6.2524857541249315, -75.5686261812429),
 (6.231704691813073, -75.61160925921747),
 (6.290384323934336, -75.56945687270888),
 (6.210362095508166, -75.57088108434691),
 (6.2451496127526225, -75.57371337731854),
 (6.271208962002754, -75.56556245591827)]

Se crea una variable para mostrar los nombres de las ubicaciones de cada uno de los nodos a los que se debe dirigir cierta ruta.

In [28]:
contenedor_labels = [contenedor['name'] for contenedor in contenedores_filtrados]
names = [depot['name']] + contenedor_labels
names

['Relleno Sanitario La Pradera',
 'Plaza Botero',
 'Universidad de Medellín',
 'Parque Juanes de la Paz',
 'Parque del Poblado',
 'Alcaldía de Medellín',
 'Parque Explora']

### Crear los datos para las Restricciones de Capacidad
Se crea una lista correspondiente a los datos necesarios para las restricciones de capacidad, para este caso las demandas de cada uno de los puntos de recogida (contenedores de basura, en este caso se simularon datos de peso dummy). Se debe tener presente que las unidades de capacidad deben coincidir con las de los vehículos.

In [29]:
demands = [0] + [contenedor['weight'] for contenedor in contenedores_filtrados]

### Graficar en el Mapa el depósito y las ubicaciones de recogida de basura
En este paso solo se tienen en cuenta los contenedores filtrados anteriormente, así que se grafican los contenedores que superen el threshold determinado y el depósito de basura.

In [30]:
m1 = fm.Map(location=coordenadas_girardota, tiles='openstreetmap', zoom_start=12)

for node in contenedores_filtrados:
    tooltip = fm.map.Tooltip("<h4><b>{}</b></p><p>Nivel: <b>{}</b></p><p>Peso: <b>{}</b></p>".format(node['name'], node['level'], node['weight']))
    ttdep = fm.map.Tooltip("<h4><b>{}</b></p>".format(depot['name']))
    mk(location=node['location'], tooltip=tooltip, icon=(fm.Icon(color='gray', icon='trash', prefix='fa'))).add_child(fm.Popup('{}'.format(node['name']))).add_to(m1)
    

mk(location=depot['location'], tooltip=ttdep, icon= (fm.Icon(color='red', icon='industry', prefix='fa'))).add_child(fm.Popup('{}'.format(depot['name']))).add_to(m1)
m1

### Construir la Matriz de Distancia
Para poder conocer las rutas optimizadas entre los puntos de recogida y el depósito, se debe crear una matriz de distancias entre estos puntos, para ello se utiliza la [**Matrix API**](https://openrouteservice.org/dev/#/api-docs/matrix) de Open Route Service.

Para poder hacer uso de la Matrix API de Open Route Service es necesario pasar los valores de las coordenadas como puntos de longitud, latitud, por lo cual se utiliza la siguiente list comprehension para obtener una variable que guarde las coordenadas invertidas agregando la coordenada del depósito para posteriormente ingresar estos valores en el modelo.

In [31]:
locations = [depot['location']] + contenedor_locations
loc_long_lat = [(x[1], x[0]) for x in locations]
loc_long_lat

[(-75.25502818729257, 6.5171946910919),
 (-75.5686261812429, 6.2524857541249315),
 (-75.61160925921747, 6.231704691813073),
 (-75.56945687270888, 6.290384323934336),
 (-75.57088108434691, 6.210362095508166),
 (-75.57371337731854, 6.2451496127526225),
 (-75.56556245591827, 6.271208962002754)]

Se procede a usar la API de Open Route Service para obtener la matriz de distancias.

In [32]:
import pandas as pd
from openrouteservice.distance_matrix import distance_matrix

request = {'client': client,
           'locations': loc_long_lat,
           'profile': 'driving-car',
           'metrics': ['distance','duration']}

try:
    response = distance_matrix(**request)
    
    print("Se calcularon {}x{} rutas.".format(len(response['durations']), len(response['durations'][0])))

    dm_durations = pd.DataFrame(response['durations'])
    dm_distances = pd.DataFrame(response['distances'])

except:
    print('Algo salió mal durante la construcción de la matriz de distancias.')
dist_matrix = response['distances']
dm_distances

Se calcularon 7x7 rutas.


,0,1,2,3,4,5,6
0,0.00,52921.63,57090.13,48601.61,58155.85,53777.19,51568.20
1,53829.31,0.00,7036.92,5277.86,5523.06,1752.96,2413.59
2,59978.32,7381.68,0.00,9434.38,7025.55,5696.77,8611.03
3,49422.39,5092.44,9863.00,0.00,9962.40,5840.75,3483.86
4,59304.22,5384.52,7582.69,11048.48,0.00,5022.67,7936.93
5,55119.02,1531.39,6178.21,6567.57,4388.75,0.00,3751.73
6,52037.30,2371.27,9260.51,3468.03,7894.33,4073.99,0.00


Se procede a usar numpy para redondear los valores de la matriz de distancia, ya que OR-Tools utiliza valores enteros para sus cálculos, se verifica que quede tal cual como se necesita.

In [33]:
import numpy as np
dist_matrix = np.array(dist_matrix)
dist_matrix = np.rint(dist_matrix)
dist_matrix = dist_matrix.astype(int)
dist_matrix = dist_matrix.tolist()
dist_matrix

[[0, 52922, 57090, 48602, 58156, 53777, 51568],
 [53829, 0, 7037, 5278, 5523, 1753, 2414],
 [59978, 7382, 0, 9434, 7026, 5697, 8611],
 [49422, 5092, 9863, 0, 9962, 5841, 3484],
 [59304, 5385, 7583, 11048, 0, 5023, 7937],
 [55119, 1531, 6178, 6568, 4389, 0, 3752],
 [52037, 2371, 9261, 3468, 7894, 4074, 0]]

### Lógica de Solución
Con la Matriz de Distancia obtenida se procede a solucionar el problema haciendo uso de OR-Tools. El solver de VRP es un componente de OR-Tools (https://developers.google.com/optimization/routing/vrp).  Documentación adicional en relación al algoritmo y las opciones de computación están disponibles en OR-Tools.

In [34]:
"""Vehicles Routing Problem (VRP)."""
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(distance_matrix, num_vehicles, demands, vehicle_capacities):
    """Se guardan los datos para el problema."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['demands'] = demands
    data['vehicle_capacities'] = vehicle_capacities    
    data['num_vehicles'] = num_vehicles
    data['depot'] = 0
    data['names'] = names
    return data

def extract_routes(num_vehicles, manager, routing, solution):
    routes = {}
    optimal_coords = {}
    for vehicle_id in range(num_vehicles):
        routes[vehicle_id] = []
        optimal_coords[vehicle_id] = []
        index = routing.Start(vehicle_id)
        while not routing.IsEnd(index):
            routes[vehicle_id].append(manager.IndexToNode(index))
            optimal_coords[vehicle_id].append(loc_long_lat[manager.IndexToNode(index)])
            index = solution.Value(routing.NextVar(index))
        routes[vehicle_id].append(manager.IndexToNode(index))
        optimal_coords[vehicle_id].append(loc_long_lat[manager.IndexToNode(index)])
    return routes, optimal_coords

def print_solution(data, manager, routing, solution):
    """Imprime las soluciones en la consola."""
    #print(f'Objectivo: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Ruta para el vehículo {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += '|{0}({1}) [Carga({2})]| -> '.format(data['names'][node_index], node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        node_index = manager.IndexToNode(index)
        plan_output += ' |{0}({1})  [Carga({2})]|\n'.format(data['names'][node_index], manager.IndexToNode(index), route_load)
        plan_output += 'Distancia de la ruta: {}m\n'.format(route_distance)
        plan_output += 'Carga de la ruta: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Distancia total de todas las rutas: {}m'.format(total_distance))
    print('Carga total de todas las rutas: {}'.format(total_load))

def solve_vrp(distance_matrix, num_vehicles, demands, vehicle_capacities):
    # Instanciamiento el problema de datos.
    data = create_data_model(distance_matrix, num_vehicles, demands, vehicle_capacities)

    # Crea el administrador de índice de enrutamiento.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Crea el modelo de enrutamiento.
    routing = pywrapcp.RoutingModel(manager)

    # Se crea y registra un callback de tránsito.
    def distance_callback(from_index, to_index):
            """Devuelve la distancia entre dos nodos."""
            # Convierte de índice de ruteo variable a NodeIndex de matriz de distancia.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define el costo de cada arco.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Se añade la restricción de capacidad.
    def demand_callback(from_index):
            """Devuelve la demanda del nodo."""
            # Convierte de índice de ruteo variable a NodeIndex de demandas.
            from_node = manager.IndexToNode(from_index)
            return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
        
    # Se añade la restricción de capacidad del vehículo al modelo
    routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            data['vehicle_capacities'],  # capacidades máximas de los vehículos
            True,  # start cumul to zero
            'Capacity')
                
    # Se añade una restricción de distancia.
    dimension_name = 'Distance'
    flattened_distance_matrix = [i for j in data['distance_matrix'] for i in j]
    max_travel_distance = 2 * max(flattened_distance_matrix)

    routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            max_travel_distance,  # distancia máxima de viaje del vehículo
            True,  # start cumul to zero
            dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
         

    # Configuración de la primera heurística de solución.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    #search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    #search_parameters.time_limit.FromSeconds(10)

    # Solucionar el problema.
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(data, manager, routing, solution)
        routes = extract_routes(num_vehicles, manager, routing, solution)
        return routes
    else:
        print('No se encontró solución.')  


### Resolver para Distancia
La ruta generada para cada vehículo se registra en consola, inlcuido un diagrama nodo a nodo. Se añaden las distancias y la carga recogida por cada vehículo.

In [35]:
try:
    routes = solve_vrp(dist_matrix, num_vehicles, demands, vehicle_capacities)
except:
    print('Algo salió mal al resolver el VRP.')

Ruta para el vehículo 0:
|Relleno Sanitario La Pradera(0) [Carga(0)]| ->  |Relleno Sanitario La Pradera(0)  [Carga(0)]|
Distancia de la ruta: 0m
Carga de la ruta: 0

Ruta para el vehículo 1:
|Relleno Sanitario La Pradera(0) [Carga(0)]| -> |Universidad de Medellín(2) [Carga(8)]| -> |Parque Juanes de la Paz(3) [Carga(17)]| ->  |Relleno Sanitario La Pradera(0)  [Carga(17)]|
Distancia de la ruta: 115946m
Carga de la ruta: 17

Ruta para el vehículo 2:
|Relleno Sanitario La Pradera(0) [Carga(0)]| -> |Alcaldía de Medellín(5) [Carga(7)]| -> |Parque del Poblado(4) [Carga(14)]| -> |Plaza Botero(1) [Carga(24)]| -> |Parque Explora(6) [Carga(33)]| ->  |Relleno Sanitario La Pradera(0)  [Carga(33)]|
Distancia de la ruta: 118002m
Carga de la ruta: 33

Distancia total de todas las rutas: 233948m
Carga total de todas las rutas: 50


### Graficar en el mapa la solución
Para graficar la solución se hace uso de la API de direcciones de Open Route Service, para usarla se necesitó modificar la función de extracción de rutas del VRP para obtener también las coordenadas de cada ubicación.

In [36]:
routes[1]

{0: [(-75.25502818729257, 6.5171946910919),
  (-75.25502818729257, 6.5171946910919)],
 1: [(-75.25502818729257, 6.5171946910919),
  (-75.61160925921747, 6.231704691813073),
  (-75.56945687270888, 6.290384323934336),
  (-75.25502818729257, 6.5171946910919)],
 2: [(-75.25502818729257, 6.5171946910919),
  (-75.57371337731854, 6.2451496127526225),
  (-75.57088108434691, 6.210362095508166),
  (-75.5686261812429, 6.2524857541249315),
  (-75.56556245591827, 6.271208962002754),
  (-75.25502818729257, 6.5171946910919)]}

In [37]:
routes[0]

{0: [0, 0], 1: [0, 2, 3, 0], 2: [0, 5, 4, 1, 6, 0]}

In [44]:
from openrouteservice.directions import directions


request2 = {'client': client,
            'coordinates': routes[1],
            'profile': 'driving-car',
            'format_out': 'geojson'}

def style_function(color):
    return lambda feature: dict(color=color, weight=5, opacity=0.7)

def map_solution_fm(routes):
    
    colors = ['blue','red','green','#0000ff','#ff0000','#00ff00']
    optimal_route = []
    
    for vehicle_id in routes[0]:
        waypoints = []
        
        request2['coordinates'] = routes[1][vehicle_id]
        optimal_route.append(directions(**request2))
        
        for contenedor_index in routes[0][vehicle_id][1:-1]: 
                waypoints.append(contenedores[contenedor_index-1]['location'])
            
        if len(waypoints) == 0:
            print('Vehículo {}: Ruta vacía\n'.format(vehicle_id))
        
        else:
            distancia = optimal_route[vehicle_id]['features'][0]['properties']['summary']['distance']
            duracion = optimal_route[vehicle_id]['features'][0]['properties']['summary']['duration']
                    
            gj = fm.features.GeoJson(data=optimal_route[vehicle_id], name='Vehículo {}'.format(vehicle_id), style_function=style_function(colors[vehicle_id]), overlay=True)
            
            gj.add_child(fm.Tooltip(
                """<h4>Vehículo {vehicle}</h4>
                <b>Distancia:</b> {distance} m <br>
                <b>Duración:</b> {duration} secs
                """.format(vehicle=vehicle_id, distance=distancia, duration=duracion))) 
                
            gj.add_to(m1)
            print('Vehículo {}:\n Distancia: {} m\n Duración : {} secs\n'.format(vehicle_id, distancia, duracion))
              
    
if routes:
    map_solution_fm(routes)    
    
else:
    print('No se encontró solución.') 
    
fm.LayerControl().add_to(m1)
m1.save('mapa_vrp_ors+ortools.html')   
m1

Vehículo 0: Ruta vacía

Vehículo 1:
 Distancia: 115946.9 m
 Duración : 7242.599999999999 secs

Vehículo 2:
 Distancia: 118001.3 m
 Duración : 7210.7 secs

